In [288]:
from delphi_epidata import Epidata
import pandas as pd

In [289]:
df = pd.DataFrame()

In [290]:
# Data has to be acquired in batches since there is a lenght limit in the API

# 8/04/2020 - 20/04/2020
res = Epidata.covidcast('fb-survey', 'raw_wcli', 'day', 'county', [Epidata.range(20200408, 20200420)], geo_value='*')
print(res['result'], res['message'], len(res['epidata']))
df = df.append(res['epidata'])

1 success 2928


In [291]:
# 21/04/2020 - 5/05/2020
res = Epidata.covidcast('fb-survey', 'raw_wcli', 'day', 'county', [Epidata.range(20200421, 20200505)], geo_value='*')
print(res['result'], res['message'], len(res['epidata']))
df = df.append(res['epidata'])

1 success 3430


In [292]:
# 6/05/2020 - 20/05/2020
res = Epidata.covidcast('fb-survey', 'raw_wcli', 'day', 'county', [Epidata.range(20200506, 20200520)], geo_value='*')
print(res['result'], res['message'], len(res['epidata']))
df = df.append(res['epidata'])

1 success 3232


In [293]:
# 21/05/2020 - 27/05/2020
res = Epidata.covidcast('fb-survey', 'raw_wcli', 'day', 'county', [Epidata.range(20200521, 20200527)], geo_value='*')
print(res['result'], res['message'], len(res['epidata']))
df = df.append(res['epidata'])

1 success 1867


In [294]:
counties = []
for county in df['geo_value']:
    counties.append(county)
counties = list(set(counties))
cleanedDf = pd.DataFrame(dtype=str)

In [295]:
for county in counties:
    countyData = df[df['geo_value'] == county]
    countyData = countyData.reset_index()
    countyData = countyData.drop(['index'], axis=1)
    
    # Time series is in rows unlike applemobility data, converting to columns instead
    temp = pd.DataFrame(columns=['geo_value', *countyData['time_value']])
    
    # Renaming columns to match applemobility data
    columns = ["geo_value"]
    for i,column in enumerate(temp.columns):
        if i == 0:
            continue
        column = list(str(column))
        column.insert(4, '-')
        column.insert(7, '-')
        columns.append(''.join(column))
    temp.columns = columns
    
    values = []
    for i,column in enumerate(columns):
        if i == 0:
            values.append(str(countyData['geo_value'].values[0]))
            continue
        else:
            columnAdj = int(column.replace('-', ''))
            value = countyData[countyData['time_value'] == columnAdj].values[0][3]
            values.append(value)
    
    temp = temp.append(pd.Series(values, index=temp.columns), ignore_index=True)
    cleanedDf = cleanedDf.append(temp, ignore_index=True)

In [296]:
cleanedDf = cleanedDf.reindex(sorted(cleanedDf.columns), axis=1)
cleanedDf = cleanedDf.fillna(0)
firstCol = cleanedDf.pop('geo_value')
cleanedDf.insert(0, 'geo_value', firstCol)

In [298]:
cleanedDf.to_csv('covidsymptoms_fb.csv')